# Managing Machine Learning Models with MLflow


## Introduction
- Weston Bassler ML Engineer at Emburse
- DevOps / SRE 
- Distributed Systems Background (Hadoop, Apache Mesos, Kubernetes)
- Love Tech and Automation


## What am I going to show you today?
- What is MLflow?
- Experiment Tracking
- Model Evaluation
- Model Registry
- Model Deployment
- Automate Training

** Please use the Notebook to follow along **


# What is MLflow?
- A powerful Python Library that assists many steps of the the Machine Learning (ML) lifecycle
- It contains of Tracking, Models, Model Registry and Model Serving
- Has integrations with many tools and platforms such as: PyTorch, Tensorflow, scikit-lear, HuggingFace, LangChain, OpenAI and many many more...
- It is Open Source

# Why should you care?

*The Application Development Lifecycle is hard. Application Development that includes Machine Learning (ML Lifecycle) is even harder!*

MLflow simplies the ML Lifecycle by providing solutions for the following:

- Experimentation Management (**MLflow Tracking**): MLflow provides a systematic way to track experiments, including parameters, metrics, and code versions. This helps you organize and compare different experiments easily, leading to more efficient exploration of hyperparameters and model architectures.

- Reproducibility and Collaboration (**MLflow Tracking & MLflow Projects & Model Registry**): MLflow captures the environment and dependencies for each experiment, ensuring reproducibility across different environments. This is crucial for collaboration within teams and sharing results with stakeholders, as it ensures that experiments can be replicated reliably.

- Model Versioning and Management (**Model Registry**): MLflow allows you to version models, making it easier to track changes over time and revert to previous versions if needed. This enhances model governance and facilitates auditing and compliance requirements.

- Deployment Simplification (**Model Registry**): MLflow streamlines the process of deploying models into production by providing tools for packaging models in a standard format and integrating with deployment platforms. This reduces the friction between model development and deployment, enabling faster time-to-market for machine learning solutions.

- Integration with Existing Tools and Frameworks (**MLflow Models/Flavors**): MLflow seamlessly integrates with popular machine learning libraries and frameworks, including TensorFlow, PyTorch, scikit-learn, and XGBoost. This means you can continue using your preferred tools while benefiting from MLflow's capabilities.

# Using MLflow Python API

#### MLflow Module
- The `mlflow` module is an for managing MLflow Runs.
- What is a Run? Collection of parameters, metrics, artifacts, etc.. related to training an ML Model
- "Active"
```py
import mlflow

# to start a run
mlflow.start_run()

# to end a run
mlflow.end_run()

```

#### MLflow Client
- Used to interface with Experiments, Runs, Model Versions and Registered Models.
```py
from mlflow import MlflowClient

client = MlflowClient()

```

## Getting Started

In [ ]:
!pip install mlflow==2.8.1 pandas==1.5.1 xgboost==1.6.2 argparse

In [ ]:
# IF YOU ARE USING GOOGLE COLAB UNCOMMENT AND RUN
# !git clone https://github.com/geekbass/MLflow-Workshop.git
# !rm MLflow-Workshop/*ipynb
# !mv MLflow-Workshop/* .

In [ ]:
# IF YOU ARE USING GOOGLE COLAB UNCOMMENT AND RUN
# import shutil

# shutil.rmtree('MLflow-Workshop')

In [ ]:

"""
RESTART YOUR KERNEL
"""


# Project Background
We are an investment firm that finanically backs startup tech companies. The company believes that they could benefit from an ML model that could estimate the potential profit based on historical data from previous investments.

We have a dataset that we have collected that estimates the potenital profit of a startup based on the spend of R&D, Administration, and Maketing as well as which U.S. state the startup will reside. 

We have decided that the model should be a Regression Type model and we are going to use XGBoost Regressor.

# Create your Project in MLflow

In [ ]:
# Create Your Initial Project in MLflow
import mlflow 

# Set the a tracking URI to a local sqlite file
mlflow.set_tracking_uri("sqlite:///mydb.sqlite")

# In MLflow create a new Experiment 
experiment_id = mlflow.create_experiment("PotentialStartups")

In [ ]:
# Print the Experiment Name and Creation Date
experiment = mlflow.get_experiment(experiment_id)
print("Name: {}".format(experiment.name))
print("Creation timestamp: {}".format(experiment.creation_time))

In [ ]:
# Run an MLFlow UI for a Visual
!mlflow server --backend-store-uri="sqlite:///mydb.sqlite"

# Track your First Model | MLflow Tracking 
Here we are going to create a project so that we can store and log information about our training runs to MLflow. MLflow Tracking provides a central location for visualizations and storing information about models such as training parameters, metrics, and even store files such as models, code, etc...

In [ ]:
# Begin with loading the Dataset into Training Data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load the Dataset
df = pd.read_csv('startups_profit.csv', index_col=False)
df['State']=df['State'].map({'New York':0,'Florida':1, 'California': 2}).astype(int)

# Training Data
X = df[["R&D Spend", "Administration", "Marketing Spend","State"]]
y = df[["Profit"]]
X, y = df.iloc[:, :-1], df.iloc[:, -1] 

# Setting up train test split
X_train, X_test, y_train, y_test = train_test_split(X, np.ravel(y), train_size=0.7,random_state=0)

In [ ]:
len(X_train), len(X_test), len(y_train), len(y_test)

In [ ]:
# Log model to our Project
import mlflow

# Set the connection to the tracking URI
mlflow.____
# Set the experiment
mlflow.set_experiment("PotentialStartups")

### Lets Talk about Auto Logging in MLflow
What is `autolog`? 

MLflow has integrations with some ML libraries that will automatically log metrics, parameters, and models by simply calling `autolog()` method.

The following libraries support autologging:
- Fastai
- Gluon
- Keras
- LightGBM
- PyTorch
- Scikit-learn
- Spark
- Statsmodels
- XGBoost



In [ ]:
# Start an MLflow Run
mlflow.____

In [ ]:
# Set Autolog for XGBoost
import mlflow.xgboost

mlflow.xgboost.____

In [ ]:
# Train our First Model
import xgboost 

xgbr = xgboost.XGBRegressor() 
xgbr.fit(X_train, y_train)

In [ ]:
# Evaluate our Model using MLflow. This is Log the metrics for us to MLflow.
eval_data = X_test
eval_data["Profits"] = y_test

# This will load our Model
model_uri = mlflow.get_artifact_uri("model")

# This will run the evaluate Method against our model and our evaluation Data for the Regressor Type.
# Here we are also only selecting the "default" evaluators
result = mlflow.evaluate(
    model_uri,
    eval_data,
    targets="Profits",
    model_type="regressor",
    evaluators="default"
)

In [ ]:
# End our Run
mlflow.____

In [ ]:
# Run this Cell a few times just to populate some data
import mlflow.xgboost
import xgboost

# Start another MLflow Run
with mlflow.start_run() as run:
    mlflow.xgboost.autolog()

    xgbr = xgboost.XGBRegressor() 
    xgbr.fit(X_train, y_train)

    # Evaluate our Model using MLflow
    eval_data = X_test
    eval_data["Profits"] = y_test
    
    # This will load our Model
    model_uri = mlflow.get_artifact_uri("model")
    
    # Set the evaluation function
    result = mlflow.____(
        model_uri,
        eval_data,
        targets="Profits",
        model_type="regressor",
        evaluators="default"
    )

# Evaluate our Trained Models based on Metrics
Just like we have in the UI, you can also sift through metrics for an Experiment. We can use Pandas for this!

In [ ]:
import mlflow
import pandas as pd

# Set Tracking URL 
mlflow.set_tracking_uri("sqlite:///mydb.sqlite")

# Get the Experiment ID
experiment_id = mlflow.get_experiment_by_name("PotentialStartups").experiment_id

# Search runs and output to Pandas DF
evals_df = mlflow.search_runs([experiment_id])
evals_df.info()

In [ ]:
evals_df

In [ ]:
# Sort it by r2_score
evals_df = mlflow.search_runs([experiment_id], order_by=["metrics.r2_score DESC"])
evals_df

In [ ]:
# Print ONLY the r2_score and the run_id
evals_df[[____, ____]]

#### The above evaluation can be done on ANY metric you would like. This will help us decide which models we would like to Regster to the Model Registry

# Register A Model | MLflow Registry
The Model Registry is used as a way to store models in a way that allows for us to share models easily to others while also following the development lifecycle (Staging, Production, etc...). It also provides a way to version, alias, tag and annotate models as desired. 

We have now trained a few models and have evaluated the results. We have a model ready for initial testing and need to establish a method for team members and other company personnel to access it.

In [ ]:
# Create a New Model in The Model Registry using the MLflow Client
import mlflow

# Set out tracking URI
mlflow.set_tracking_uri("sqlite:///mydb.sqlite")

# Create a client connection
client = mlflow._____

# Create a new Model in the Registry called StartupModels
client.create_registered_model("StartupModels")

#### Now that we have a location to store our models lets register (add) a model to it

In [ ]:
import mlflow

# SET THESE 2 lines
mlflow.set_tracking_uri("sqlite:///mydb.sqlite")
mlflow.set_experiment("PotentialStartups")

In [ ]:
# To begin using the Model Registry, Pick our favorite model from above and register it using the run-id
run_id = ____

# Register the model
mlflow.register_model(f"runs:/{run_id}/model", "StartupModels")

#### Notice here, a new version of the model has been created. We could add another model here and it would continue to increment the version. This is very good practice. 

#### Also note here, MLflow increments versions but we can also add our own tags and aliases to models to better help identify them. This is out of scope for this workshop.

# Deploying a Model | MLflow Registry
Now that we have a model registered to the Model Registry, we can now use MLflow to Deploy the model. Anyone that has access to MLflow can do this. For now we are going to deploy the model locally using the version of the model when we first registered it.

We will load it via the "model_uri" - `models:/model_name/model_version`

In [ ]:
import mlflow

# Set the tracking URI
mlflow.set_tracking_uri("sqlite:///mydb.sqlite")

In [ ]:
import mlflow

# Notice here we actually use mlflow XGBoost "flavor" to load the model. Check the MLflow Docs for more information on Flavors!
model = mlflow.xgboost.load_model(model_uri="models:/StartupModels/1")
model

In [ ]:
# Run a quick Prediction on profit using some fake data

# R&D Spend, Administration, Marketing Spend, State
predict_list = [345349.2, 133337.8, 472345.10, 1]
# Predict
prediction = model.predict([predict_list])
prediction[0]

### Serving via CLI
Below is an example shell script using the MLflow CLI for serving the same model. It will serve the model via port 5000.

```sh
#!/usr/bin/env sh

# Deploying a Model using the mlflow cli
export MLFLOW_TRACKING_URI="sqlite:///mydb.sqlite"

# Serve the model from the 
mlflow models serve -m "models:/StartupModels/1" --no-conda
```

# Automating Training | MLflow Projects
As we iterate new model versions, add new data and learn from our experimentations it becomes more of a need to implement automation into our development pipeline. This is where MLflow Projects come in. MLflow Projects is a format used to package all of our code into a reproducible way. It also provides a way for us to run the automation via CLI or from the `projects()` function. 

We specify our Project in a file called `MLproject` which is a yaml file that specifies key pieces such as name, python environment and entry points which are ways that we can pass templated parameters and commands to scripts.

Below is an example `MLproject` file that we will be using:

```
name: Potential Profit 

python_env: python_env.yaml

entry_points:
  main:
    parameters:
      n_estimators: {type: int, default: 10}
      max_depth: {type: int, default: 5}
    command: "python train.py \
        --n_estimators {n_estimators} \
        --max_depth {max_depth}"
```

The above file will allow for us to pass the parameters of `n_estimators` and `max_depth` to our command which is a python script that accepts two arguments (n_estimators` and `max_depth`). This will help us to automate a training run where we can then pass different parameters using the same code. For this we will use Argparse library.

**We can automate an entire training run very very easily! Everything we have done so far we will automate with an MLflow Project!**


Our `train.py` file looks like this:

```py
import sys
import argparse
import mlflow
import mlflow.xgboost
import xgboost
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Set MLflow tracking server and the Experiment Name
mlflow.set_tracking_uri("sqlite:///mydb.sqlite")

# Parse out our parameter Arguments
parser = argparse.ArgumentParser()
parser.add_argument('--n_estimators')
parser.add_argument('--max_depth')
args = parser.parse_args()

# Set the values of our Arguments
n_estimators = int(args.n_estimators)
max_depth = int(args.max_depth)

# Set up the Training Data
# Load the Dataset
df = pd.read_csv('startups_profit.csv', index_col=False)
df['State']=df['State'].map({'New York':0,'Florida':1, 'California': 2}).astype(int)

# Training Data
X = df[["R&D Spend", "Administration", "Marketing Spend","State"]]
y = df[["Profit"]]
X, y = df.iloc[:, :-1], df.iloc[:, -1] 

# Setting up train test split
X_train, X_test, y_train, y_test = train_test_split(X, np.ravel(y), train_size=0.7,random_state=0)

# Start a training Run and autolog it.
with mlflow.start_run() as run:
    mlflow.xgboost.autolog()
    xgbr = xgboost.XGBRegressor(n_estimators=n_estimators, max_depth=max_depth) 
    xgbr.fit(X_train, y_train)

    # Evaluate our Model using MLflow
    eval_data = X_test
    eval_data["Profits"] = y_test
    
    # Load the model
    model_uri = mlflow.get_artifact_uri("model")
    
    # Evaluate the model and autolog it
    result = mlflow.evaluate(
        model_uri,
        eval_data,
        targets="Profits",
        model_type="regressor",
        evaluators="default"
    )

```

You can see all the steps that we have done before to train our model except we are now adding parameters to our model which is going to be passed via MLflow Projects.

In [ ]:
# Lets run a Project using the projects function
import mlflow

# Set our tracking uri
mlflow.set_tracking_uri("sqlite:///mydb.sqlite")

# Run the projects with our specified parameters
mlflow.projects.run(
    # Specifies where the MLproject file lives
    './',
    # Running this on the main entry point
    entry_point='main',
    # Here is our Experiment Name.
    experiment_name='PotentialStartups',
    # Using the local environment
    env_manager='local',
    # Set our Desired parameters for our model
    parameters={
        'n_estimators': 20, 
        'max_depth': 5
    })


In [ ]:
# Lets just check to make sure it worked
import mlflow
import pandas as pd

# Set Tracking URL 
mlflow.set_tracking_uri("sqlite:///mydb.sqlite")

# Get the Experiment ID
experiment_id = mlflow.get_experiment_by_name("PotentialStartups").experiment_id

In [ ]:
# Search runs and output to Pandas DF. You can get the run_id from the output from the Project run.
evals_df = mlflow.____([experiment_id])
evals_df['run_id']==____

## Taking Automation Further with Projects
- Use tools like Dask or Ray to train multiple Models at a time with different Parameters
- Dask Hyperparameter Search is a great option
- Train multiple models at the same time in parallel


In [ ]:
!pip install dask

In [ ]:
# Here we are creating a list of parameters to be passed to our train_model function below.
import random

parameters = [
    {'n_estimators': 10, 'max_depth': 2},
    {'n_estimators': 50, 'max_depth': 3},
    {'n_estimators': 100, 'max_depth': 4},
    {'n_estimators': 20, 'max_depth': 5},
    {'n_estimators': 150, 'max_depth': 6},
    {'n_estimators': 250, 'max_depth': 7},
    {'n_estimators': random.randint(0, 250), 'max_depth': 8},
    {'n_estimators': random.randint(0, 250), 'max_depth': 9},
    {'n_estimators': random.randint(0, 250), 'max_depth': 10},
    {'n_estimators': random.randint(0, 250), 'max_depth': 9},
    {'n_estimators': random.randint(0, 250), 'max_depth': 8},
    {'n_estimators': random.randint(0, 250), 'max_depth': 7},
    {'n_estimators': random.randint(0, 250), 'max_depth': 6},
    {'n_estimators': random.randint(0, 250), 'max_depth': 5},
    {'n_estimators': random.randint(0, 250), 'max_depth': 10}
]

In [ ]:
len(parameters)

In [ ]:
# Using the same Projects functions as above but we take the parameters from a dictionary in dask delayed decorator
import dask
import mlflow

@dask.delayed
def train_model(parameters):
    # Set our tracking uri
    mlflow.set_tracking_uri("sqlite:///mydb.sqlite")

    # Run the projects with our specified parameters
    mlflow.projects.run(
        # Specifies where the MLproject file lives
        './',
        # Running this on the main entry point
        entry_point='main',
        # Here is our Experiment Name.
        experiment_name='PotentialStartups',
        # Using the local environment
        env_manager='local',
        # Set our Desired parameters for our model
        parameters={
            'n_estimators': parameters['n_estimators'], 
            'max_depth': parameters['max_depth']
        })





In [ ]:
results = []

# Append the results for each dictionary of parameters
for param in parameters:
    results.append(train_model(param))

# Compute it in parallel
dask.compute(results)

In [ ]:
# Show all the new ones
import mlflow
import pandas as pd

# Set Tracking URL 
mlflow.set_tracking_uri("sqlite:///mydb.sqlite")

# Get the Experiment ID
experiment_id = mlflow.get_experiment_by_name("PotentialStartups").experiment_id

# Sort it by r2_score and show which parameters gave us the best result
evals_df = mlflow.search_runs([experiment_id], order_by=["metrics.r2_score DESC"])
evals_df[["metrics.r2_score", "run_id", "params.max_depth", "params.n_estimators"]]

In [ ]:
# Lets Register the "BEST" model
run_id = "a552865619f043b4a5a88bfda2738a24"

# Register the model
mlflow.register_model(f"runs:/{run_id}/model", "StartupModels")

In [ ]:
# Load and run prediction on BEST model
import mlflow

# Load Second version
model = mlflow.xgboost.load_model(model_uri="models:/StartupModels/2")
model

In [ ]:
# print params
model.max_depth, model.n_estimators

In [ ]:
# R&D Spend, Administration, Marketing Spend, State
predict_list = [13345349.2, 200000.8, 100000.0, 0]
# Predict
prediction = model.predict([predict_list])
prediction[0]